# Machine Learning Model Creation in Splice Machine
#### Starting the Spark Session

In [ ]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

In [ ]:
# Setup
from pyspark.sql import SparkSession
from splicemachine.spark import PySpliceContext
from splicemachine.mlflow_support.utilities import get_user

spark = SparkSession.builder.getOrCreate()
splice = PySpliceContext(spark)
schema = get_user()

## Importing MLflow Support
<blockquote><p class='quotation'><span style='font-size:15px'> As explained in <a href='./7.2 Splice MLflow Support.ipynb'>7.2 Splice MLflow Support</a>, using MLflow on Splice Machine is extremely easy. Check out our <a href='https://pysplice.readthedocs.io/en/latest/splicemachine.mlflow_support.html'>documentation</a> for the available functionality.<br><footer>Splice Machine</footer>
</blockquote>

In [ ]:
# MLFlow Setup
from splicemachine.mlflow_support import *
mlflow.register_splice_context(splice)

## Starting an experiment
<blockquote><p class='quotation'><span style='font-size:15px'> Here we'll begin an experiment to keep track of our modeling efforts for this prediction task.<footer>Splice Machine</footer>
</blockquote>

In [ ]:
mlflow.set_experiment('model_creation_demo')

## Starting a run
<blockquote><p class='quotation'><span style='font-size:15px'> Here we'll begin an experiment to keep track of our modeling efforts in this notebook specifically.<footer>Splice Machine</footer>
</blockquote>

In [ ]:
#start our first MLFlow run
from datetime import datetime

tags = {'team': 'Splice Machine', 'purpose': 'fraud DEMO'}
mlflow.start_run(tags=tags, run_name=f"RF_run")

## Ingesting Data
<blockquote><p class='quotation'><span style='font-size:15px'> Ingesting the table created in <a href='./7.3 Data Exploration.ipynb'>7.3 Data Exploration</a>, we will begin constructing a very simple Machine Learning Model. <footer>Splice Machine</footer>
</blockquote>

In [ ]:
sql_query = f"SELECT * FROM {schema}.cc_fraud_data"
df = splice.df(sql_query)

## Logging our first Parameter 
<blockquote><p class='quotation'><span style='font-size:15px'> We're utilizing MLFlow to keep track of the query we used to ingest the data for this modeling effort. <footer>Splice Machine</footer>
</blockquote>

In [ ]:
# Logging our first parameter: the query we used to ingest our data
mlflow.log_param("ingest_query", sql_query)

## Selecting Our Features
<blockquote>Here we'll select the features only most strongly correlated to our target<footer>Splice Machine</footer>
</blockquote>

In [ ]:
import pandas as pd
pdf = df.filter(df.CLASS_RESULT == 0).limit(900).toPandas()\
        .append(df.filter(df.CLASS_RESULT == 1).limit(100).toPandas())
pdf = pdf.apply(pd.to_numeric)
corr = pdf.corr()

most_correlated = corr.abs()['CLASS_RESULT'].sort_values(ascending=False).reset_index()
most_correlated = most_correlated.iloc[1:].rename({"index":"feature","CLASS_RESULT":"correlation_to_target"}, axis = 1)
print(most_correlated)

In [ ]:
CORRELATION_CUTOFF = 0.05
#Logging this in mlflow
mlflow.log_param("correlation_cutoff", CORRELATION_CUTOFF)

feature_cols = list(most_correlated[most_correlated['correlation_to_target']>CORRELATION_CUTOFF]['feature'])
print(feature_cols)

## Defining a Machine Learning Pipeline

<blockquote>We'll use Spark's <code>Pipeline</code> class to define a set of <code>Transformers</code> that get your dataset ready for modeling<br>
We'll then use <code>mlflow</code> to <code>log</code> our Pipeline stages. Both <code>log_pipeline_stages</code> and <code>log_feature_transformations</code> are custom Splice Machine functions for tracking Spark Pipelines. </blockquote>

In [ ]:
%%time
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.classification import RandomForestClassifier, MultilayerPerceptronClassifier

"""
The preprocessing stages for this example are: 
1) Vector assembling the feature columns 
2) Standardizing our feature columns
"""
max_depth = 5  
num_trees = 20

assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
scaler = StandardScaler(inputCol="features", outputCol='scaledFeatures')
rf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'CLASS_RESULT', maxDepth = max_depth, numTrees = num_trees)

# Pipeline to preprocess and model our data
mlpipe = Pipeline(stages=[assembler,scaler, rf])

# Custom Splice functions to add granularity and governance to your Spark Pipeline Models
mlflow.log_pipeline_stages(mlpipe)
mlflow.log_feature_transformations(mlpipe)

## Separating our data for performance evaluation 
<blockquote> We are using a simple, single train/ test split to assess the performance of our simple model. Of note, we are not invesitgated the class balances, and we are using untuned hyperparameters to predict the target variable. These can be adjusted as an exercise. <footer>Splice Machine</footer>
</blockquote>

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from splicemachine.stats import *

# Split our data into a training and testing set
train, test = df.randomSplit([0.8, 0.2])

mlflow.lp("train_ratio", 0.80)

## Fitting our model 
<blockquote> Training our model and logging executing time using Splcie's custom <code>with mlflow.timer('timer_name')</code> block function to track the time it takes to complete a block. Everything in the block will be timed, and then logged to mlflow under the timer name provided to the function. <footer>Splice Machine</footer>
</blockquote>

In [ ]:
with mlflow.timer('training'):
    fitted_model = mlpipe.fit(train)
# Log the parameters of the best model
mlflow.log_model_params(fitted_model)


## Assessing our Model Performance
<blockquote> Making predicitons on the test set, evaluating performance, and logging this to MLFlow <footer>Splice Machine</footer>
</blockquote>

In [ ]:
#Inference
predictions = fitted_model.transform(test)

#Performance Evaluation
binary_evaluator = SpliceBinaryClassificationEvaluator(spark, labelCol = "CLASS_RESULT")
binary_evaluator.input(predictions)
performance_metrics = binary_evaluator.get_results(as_dict = True)

#Logging Performance
mlflow.log_metrics(performance_metrics)

## Logging Artifacts of this Run
<blockquote> We can store the notebook associated with a particular run as well as the fitted model created by this run <footer>Splice Machine</footer>
</blockquote>

In [ ]:
# Store the notebook for easy retrieval
mlflow.log_artifact('7.5 Model Creation.ipynb', 'training_notebook')
#Log the best model
mlflow.log_model(fitted_model, 'rf_model')

## Finish our run
<blockquote>Now we'll end our run, and view the results in the <a href="/mlflow">MLFlow UI</a>. We can look at our different runs, the parameters, metrics, tags and artifacts logged, and download our notebook directly. You'll know the run is complete fom the small green check mark on the leftmost side of the run</blockquote>

In [ ]:
from splicemachine.notebook import get_mlflow_ui
get_mlflow_ui(mlflow.current_exp_id(), mlflow.current_run_id())

In [ ]:
mlflow.end_run()

In [ ]:
spark.stop()

# Fantastic!
<blockquote> 
This basically shows how our platform can be used to train and evaluate machine learning models! <br>
    Next Up: <a href='./7.6 Model Deployment.ipynb'>Using MLManager to Deploy Machine Learning Models</a>
<footer>Splice Machine</footer>
</blockquote>